# Introduction to QoS

__Prerequisits__

* Packets and header structures 
* Life cycle of a packet/frame 
* Familiarity with IOS CLI 
* Basics of Routing and Switching

__Agenda__

* __What is QoS and why we need it? :__ What problem does this feature solve 
* __QoS based tools :__ Frameworks such as MQC, HQF 
* __Classification and Marking :__ How to control the priotitization of traffic in a congested network. (IP-Precedence, DSCP, NBAR etc.)
* __Congestion Management__: How to control the piling up of data in an interface-buffer 
* __Queue management__ : 
* __Congestion Avoidance__ : Proactively take action before the memory gets filled up (WRED, WTD).  

# Lab Setup

## Topology

![](qos_figs/topo.png)

## Router Base Config

__Router 1__

```
!r1
conf t
	int f0/0
		ip add 12.0.0.1 255.255.255.0
		no sh
	int l0
		int l0
			ip add 192.168.1.1 255.255.255.0
	router eigrp 100
		no auto
		pass def
		net 0.0.0.0
		no pass f0/0
end 
```
__Router 2__

```
!r2
conf t
	int f0/0
		ip add 12.0.0.2 255.255.255.0
		no sh
	int s2/0
		ip add 23.0.0.2 255.255.255.0
		no sh 
	int l0
		int l0
			ip add 192.168.2.1 255.255.255.0
	router eigrp 100
		no auto
		pass def
		net 0.0.0.0
		no pass f0/0
		no pass s2/0
end 
```

__Router 3__

```
!r3
conf t
	int f0/0
		ip add 34.0.0.3 255.255.255.0
		no sh
	int s2/0
		ip add 23.0.0.3 255.255.255.0
		no sh 
	int l0
		int l0
			ip add 192.168.3.1 255.255.255.0
	router eigrp 100
		no auto
		pass def
		net 0.0.0.0
		no pass f0/0
		no pass s2/0
end 
```

__Router 4__

```
!r4
conf t
	int f0/0
		ip add 34.0.0.4 255.255.255.0
		no sh
	int f0/1
		ip add 45.0.0.4 255.255.255.0
		no sh 
	int l0
		int l0
			ip add 192.168.4.1 255.255.255.0
	router eigrp 100
		no auto
		pass def
		net 0.0.0.0
		no pass f0/0
		no pass f0/1
end
```

__Router 5__

```
!r5
conf t
	int f0/0
		ip add 45.0.0.5 255.255.255.0
		no sh
	int f0/1
		ip add 10.0.4.1 255.255.255.0
		no sh 
	int l0
		int l0
			ip add 192.168.5.1 255.255.255.0
	exit
	
	ip dhcp excl 10.0.4.1 10.0.4.100 
	ip dhcp pool LOCAL_POOL
		network 10.0.4.0 255.255.255.0
		def 10.0.4.1
	
	router eigrp 100
		no auto
		pass def
		net 0.0.0.0
		no pass f0/0
end
```

## Ostinato Setup

* Use any interface but __eth0__ (Management Interface) for connecting the router.
* Open Ostinato $\rightarrow$ select generator interface (__eth1__) $\rightarrow$ file $\rightarrow$ New Steam $\rightarrow$ Set the following settings 
* __Name__ : any, e.g. QoS_TEST
* __Protocol Section__
    * L1 = Mac
    * VLAN = Untagged 
    * L2 = Ethernet II
    * L3 = IP
    * L4 = UDP
    * L5 = None
    * Keep the rest as default 
* __Protocol Data__
    * MAC
        * Source: Any (default : 00:00:00:00:00:00)
        * destination: Routers interface use `sh int IFACE | in bia` to get the MAC
    * IPv4
        * Source: any (e.g. 10.0.1.2, just keep on the same subnet)
        * Dest: Far end interface IP, 10.0.4.1
        * TOS: override the default TOS (00) to 60 (will be explained later) 
* __Stream Control__
    * Rate = 100,000 bits/sec. This will auto calculate the packets/sec 
    * After Stream action = Goto First (to make it a indefinite loop)
* __Save Settings__: Save $\rightarrow$ Apply $\rightarrow$ Transmit start (play button)
    *  By default Ostinato sends streams sequentially if multiple streams are configured. To make is simultainious, select the interface (port) you want to run in parallel $\rightarrow$ Port configuration $\rightarrow$ interleaved streams. 
* __Verification:__ Go to the far end router i.e. R5 and check the input rate of f0/0 using `sh int f0/0 | in input rate` commnad. IOS by default uses 5 miniutes moving average based load calculation, you may change it using `load interval 30` i.e. 30 sec (minimum) load interval in the interface level. Notice the rate in not 100kbps (due to the slow serial link)

```
sh int f0/0 | in input rate
30 second input rate 56000 bits/sec, 112 packets/sec
```

Also, check the CPU throtolling histoty using `sh processes cpu history` command

```
100                                                             
 90                                                             
 80                                                             
 70                                                             
 60                                                             
 50                                                             
 40                                                             
 30                                                             
 20 **************     *****************************************
 10 ************************************************************
   0....5....1....1....2....2....3....3....4....4....5....5....6
             0    5    0    5    0    5    0    5    0    5    0
               CPU% per second (last 60 seconds)

                                                                
    
100                                                       *     
 90                                                       *     
 80                                                       *     
 70                                                   *   *     
 60                                                   *   *     
 50                                                   *   *     
 40                                                   *   *     
 30                                                   *   *     
 20 ######*                                           *   *     
 10 #######                  * *                      # **#     
   0....5....1....1....2....2....3....3....4....4....5....5....6
             0    5    0    5    0    5    0    5    0    5    0
               CPU% per minute (last 60 minutes)
              * = maximum CPU%   # = average CPU%
```

# What is QoS ? 

## Basics 

* QoS stands for Quality of Service 
* What problem does it solve?
    * Provides __predictable management of network resources__ during times of congestion.
    * assist in __maximising the end-user experience__ of critical sessions. A email traffic may be delayed but not a voice/video traffic.  
    * provides __differentiated services__ to packets based upon pre-defined user criteria. User may define which traffic is more critical than other. 
* How does QoS provide those services?
    * There are many QoS features. Some features only performs single job, others do multiple jobs.
    * The core tasks that a QoS feature may accomplish can be as follows. Some features performs one one, some does multiple.
        * __Classification of Data__: Segrigation os streams from an unified traffic. 
        * __Queue Management__: Size, placement of packets (which packet goes to which queue), Scheduling order, Transmission Rate (Traffic Shapping). 
        * __Preemptive drops__: rate limit multiple ingress interfaces to prevent an egress interface to get overloaded.   
        * __Marking of packets__: Mark packets based on its class to tell the router to treat them with different priority.  
* __Quality of Experience (QoE)__: The result an user experience as an end-to-end consequece of QoS policies. 

### Packet life cycle 

__Packer life cycle in a Router__

* packet arrived on the ingress interface (RX-RING). Interface has a DMA access to certain part of the memory called RX-RING.
* Packet queued in the memory buffer. Memory sends HW-interrupt to the CPU and relinquish control of the memory location in RX-RING to the CPU. Based on different switching algorithm (Process, Fast, CEF) router processes it.
* RIB/FIB lookup is done and forwording decission is made. 
* Packet header is manipulated and it is placed on the Tx-RING. 
* CPU relinqished the control of the TX-RING memory location to the Egress interface and the interface puts it on the line. 

The avobe process is mainly used in a roter. However in a switch, the process is different. Memory architecture of a switch could be of two types

* __Shared Memory__: In smaller switches e.g. Catalist 3500/4500, the interfaces and the ASIC (Forwarding engine) share a central TCAM memory block via a memory manager (MMU). A schedular makes a predictable schedule of the process. CPU does not see any of the process, everything is handled by the ASIC. CPU performs tasks like Spanning-Tree etc. 
* __Distributed Memory__: In a larger switches e.g. Catalist 6500 series, there are multiple line card (Supervisor engine or Control plane  and Interface Cards or Data Plane). Each DP line card has multiple interfaces with deficated ASIC that mainly performs simple jobs like FCS chek. Each interface-ASIC has their own memory (devided in Tx and Rx memory). All these individual ASIC has an access to the upstream Supervisor. ASICs in the Supervisor Engine decides the destination address and signals the appropriate eggress ASIC to send it. The mode of communication between Supervisor and Line ASICs could be via a Bus or a Ring. 

* __Packet Life Cycle in a Shared Memory system__
    * packer arives on the ingress interface
    * interface ASIC immidiately forwards the packet into common shared memory pool.
    * forward decission is made by the forwarding ASIC 
    * memory ownership is transfered to the interface ASIC 

* __Packet Life Cycle in a Distributed Memory system__
    * Packet arrives on an ingress interface. 
    * Interface/port ASIC places it into a local Queue/Buffer.
    * Forwards the packet to the Forworking engine via a BUS or a RING.
    * Forwrding ASIC makes a forwarding decission. 
    * packet transmitted onto shared ring/bus to the egress buffer. 
    * egrees interface ASIC puts it into the line. 
    
As a core takeaway, it is clear that during the process the data must be stored temporarily somewhere in the device, typically in a buffer. The router manages this buffer with software, a switch does the same in hardware level. If there is no congestion, QoS is not needed. Commonly the Egress buffer suffers from congestion as the ingress interfaces are typically rate-limited by the ISP.  

### Buffers and Queues

* __Buffer__
    * physical memory used to store packets before and after the forwarding decisions are made.
    * On routers this same memory can be allocated to interface as ingress/egress queues.
    * Shared memeory (of which, part is allocated as buffers) is also used by lots of other CPU processes.
* __Queues__
    * On routers a queue is a logical part of the shared memory buffers.
    * On switches, individual interfaces (or line-cards) have their own memroy which is used as interface queues. 
* Buffer-Queue Configuration
    * Configuration os buffer is not normally part of QoS
    * Buffer configuration would involve modifying the quantity of buffes allowed for particuar sized packet
    * __Buffer-Tuning__ in cisco IOS is not generally recomanded.
    * QoS configuration is only applicable to queeues. 
    * QoS does not alter the physical buffer size, rather it controls how packets are treated inside a queue. 
    
### What is Congestion?
    
* During times of no congestion, QoS is not needed. Packets forwarding follows a FIFO mode in Rx-Tx ring. 
* __Egresss Congestion__: Packet forwarded to egress interface faster than Tx-RING can handle them. 
* __Ingress Congestion__: Packets arrive in multiple ingress interface faster than the forwarding engine can process them (not a typical scenario). In modern routers and switches the lookup processes are doen in hardware level, therefore it looksup millions of packets per seconds, i.e. 10~20 times faster than the line rate. 

### Result of a Congestion

* __Delay__:  Typically the intr-frame space is fixed. Delay is the __Uniformly scaled IFS__. Delay is irritating for a voice.  
* __Jitter__: Jitter is the __Ununiformly scalled IFS__. Jitter kills voice traffic. 
* __Drops__: Packets gets dropped (in a voice minor drops are not persivable but not for video). 

### QoS Protocols 

* __Intigrated Services__
    * QoS model in which entire E2E pcket for a packet is __ensured certain minimum QoS charactesistics prior to the packet transmision__.
    * Initial RFC by IETF in 1990s : RFC 1633, 2211, 2212
    * __RSVP__ used a primary protocol to setup the path.
        * Requires every node along the path to heed its reservation. 
        * Requires every node along the path to keep per-flow state. 
    * __Limitation of IS:__
        * In a heterogenious environment over an E2E communication is realistic as different devices perhaps use different protocols (especially for ISP communication)
* __Differentiated Services__
    * Designed to address the challenses of IS 
    * RFC : 2474,2597, 2598,3245 and 4594
    * The DS model describes various behavious to be adopted by each complient node (Called Per-Hop Behaviour or __PHB__)
    * No prior reservation but QoS is called when a packet hits am intermediate router.     

# QoS Tools

QoS tools are genrally classified into three high level categories
* Classification and Marking 
* Congestion management framework and 
* Congestion avoidence 

## Classification and Marking 

* Traffic must first be segrigated into "Classes"
    * A class of traffic will recive the same QoS treatment. These classes must be pre-determined during the design of the network (Business Decission). 
    * Analyse packets to differentiate flows.
* Packets are marked to that the anaylis happens only a limited number of times, usually ingress edge of a network. Markig may happen at the very first stage somewhere down the line. e.g. when some one talks over an IP Phone, the phone marks the voice traffic with a special ToS byte in the IP header. You may also configure an ACL to mark different packets differently.    
* Routers perform all the classification and marking in the software level, unlike in a switch it happens in  the hardware level. Therefore, Routes can provides DPI however, switch are hardware dependedn. Consequently, QoS in routers are slower than Routers

### Policing Shapping and Markdown 

* Betwenn ISPs and Customer there are pre-defined contracted rate (Called Commited Information Rate or CIR) which is typically defined in the SLA. 
* ISP will police ingress traffic : traffic that is not-conforming is caught by policer and it may  drop or markdown (mark it for upstream device but let it go though) those extra packets.
* Customer typically doesn't want any traffic dropped (delay is better than drop). Therefore the customer does __Traffic-Shapping__ in the egress/outbound interface that only allows a traffic that complies with the SLA and CIR. 

### Queuing 

* When egreess traffic cannot immediately be trasmitted, it is placed in an egress queue.
* A single egress interface may have multiple, associated queues differentiated by priority. 
* QoS features designed for Queuing provide control over which, classified traffic is placed into each of these queues.
* Can also preemptivly drop traffic from within queues to make room for higher-prio traffic (a web traffic can be deprioritzed compared to a Telnet when the queue is about to get congested). 

### Scheduling 

* A schedular orders a packets for processing 
* On Routers QoS queuing feature (e.g. WFQ) typically affect queuing and scheduling behaviours
* On a Switch queuing and scheduling can be separate features. 
* Traffic Shapping is a Funtion of the scheduling 

## Congestion management 

The conhestion management feature allows you to controll the congestion by termining the order in which packets are sent out and interface based on priorities assigned to those packets.

* Creation of queues
* Assignment of packetw to those queues based on the classification and marking 
* Selectively dropping packets from within queues when those queues reach pre-defined thresholds
* Scheduling of the packets ina  queueu for transmission


## Congestion avoidance 

How to manage a queue so that congestion never happens. 

| __Congestion Management__ | __Congestion avoidence__ |
|--|--|
| Control queuing and scheduling of traffic | preeptively drops traffic to avoid congestion |
| WFQ, CBWFQ, PQ, LLQ, WRR, SRR, Traffic Shapping | RED, WRED, WTD, Policing |

## Modular QoS Command-Line (MQC)

* Before MQC, QoS was configured in the interface level. 
* MQC allows feactures that apply several modules such as ACL, Class-Map etc. to perform Classificatio, Policing, etc to be configured independetly and then linked-togather as needed. 
* The 3 main components of the MQC is 
    * __Class-Map:__ Classifies a traffic (Defalut behaviour __match-all__, others are __match-any__ and __match-not__ ). It does not do anything untill it is referecend  
    
    ```
    conf t
        class-map match-all PREC3   ! creates a class-map named PREC3
            match ip precedence 3   ! matches ip packet with precedence 3 
        class-map match-all TELNET  ! creates another class-map named TELNET 
            match access-group 101  ! matches ACL 101
    end
    ```
    * __Policy-Map:__ Defins action to pursue referencing a classmap 
     
    ```
    conf t
        policy-map TO_ISP             ! creates a policy map
            class PREC3               ! refereces the class-map PREC3
                bandwidth percent 30  ! action: limit bw=30mbps
        policy-map TO_CORE            ! another policy map 
            class prec3               ! matches same class 
                set dscp af33         ! action: set the DSCP value to ad33
    end 
    ```
    * __Service-Policies:__ Applies policy map to an interfacs. Service policy is directional. 
    
    ```
    conf t
        int s0/1                          ! selects an interface
            service-policy in|out TO_ISP  ! applies policy map inbound or outbound
        int f0/0
            service-policy in|out To_CORE
    end 
    ```
    
When a packet arrives on an interface, it checks the policy map then the policy map matches the condition based on the class-map. 

__Important__ : In a switch, something that sows up in a CLI in class-map match secion does not nessesarily mean that it is supported on the hardware. You may not get a generic error like __"Fearure not Supported"__ until your try to apply a a policy map in the service policy. Therefore, make sure the feature list before a purchase is made.  

## Hierachical Queuing Framework (HQF)

__Unified configuration of Qos__: If a set of platform supports exact same features, they must be configured in exact same ways.

* Consistent queuing behaviour applied with common MQC across all main Cisco IoS software releases.
* Common funtionalities for both distributed and non-distributed implementation, providing consitency of QoS feature behaviour across all software-forwarding hardware.  
* Some legacy commands are lost. First implemented in 12.04T onwards. 

# Classification and Marking 

Classification is a way to tell the routers about the sevearity of a traffic by indentifying distictive features through classes. Most common ways to classify traffic is by

* Marking : tagging packets based on its class 
* Addressig : Segregating traffic based on its source and destination IP addresses
* Application Signature : Application aware classification using Deep packet Inspection (DPI)

## L2 Classification 

* There is no priority field in Ethernet II/802.3 header.
* Therefore a L2 frame must be encapsulated by either an ISL or 802.1q header to apply prioritiztion. both supports 3 bits  
    * ISL : 3 bits Class of Service (CoS) field 
    * Dot1q: 3 bits User-Priority field  
    * Dot1p: similar to Dot1q but VLAN ID is set to 0 only. 
* In a frame-Relay header there is a Discard-Eligible (DE) bits. If a ingress burst exceeds the CIR the extra bits are marked with DE=1, which will not be QoS guranteed. Therefore, the customer marks un-important traffic (HTTP than VoIP)  with DE=True to preserve the discard control within them. DE can be done on DLCI number as well to rate-limit based on the outbound PVC. 

## L3 Classification

* Due to limited options, Classification is not done in L2 rather in L3. Both IPv4 and IPv6 has a field called __Type of Service (ToS)__ and __Traffic Class (TC)__ respectively to manage the prioritization. 
* The __Original ToS Byte__ : `__P P P D T R 0 0__`  as per RFC791
    * __P__ : Precedence 
    * __D__ : Delay (0 = normal , 1 = high) : put on a path with as low/high delay
    * __T__ : Throughput (0 = normal , 1 = high) : put on a path with as low/high thoughput
    * __R__ : Reliability (0 = normal , 1 = high) : put on a path with as low/high reliablity 
    * __Precedence__ combination 
        * __000__ : Routine (default to normal data)
        * __001__ : Priority 
        * __010__ : Immidiate 
        * __011__ : Flash
        * __100__ : Flash override
        * __101__ : CRITIC/ECP      (default for VoIP)
        * __110__ : Internet-Control (Default for Routing protocol)
        * __111__ : Network Control 
* However very fiew applications make use the bits other than precedence bits. therefore the rest of the 5 bits are wasted and all traffic must be classified into one of the 8 classes, which is not a feasible solution. 
* To solve this problem, instead of using 3 bits for priority, why not include D,T,R and make a 6-bits priority with $2^6=64$ possible priority level. This is called __Differentiated Service Code Point (DSCP)__. 
    * The policer must be told if TOS or DSCP is used for marking. 
    
    ```
    conf t
        class-map TEST
            match [ip] precedence | dscp  ! ip = IPv4 only, none = IPv4 and IPv6
            ...
    end
    ```
 
    * DSCP Pattern __`CS CS CS AF AF X ENC ECN`__
        * __CS__ : Class selector for backward comparibility (e.g. `match cs1` = `00100000`). In a CS calue all bits but CS bits are zeroed out.
        * __AF__ : For each Class selector `CS1 - CS4` they have 3 sub classes each called Assured forwarding (AF) with the next 2 bits. (e.g. `AF32 = 0111000`). There is no AF values for Class 5,6,7. Each AF value signifies a __Drop-Probability__, within a same CS=C, AF=C1 (low DP), AF=C2 (mid DP) and AF=C3 (high) i.e. __lower AF wins__. If there are two packets of different class C1 and C2, __higher C-value wins__. However, this feature are not enabled by default rather it is recommanded in the RFC, you need to explicitly configure the behaviour.
        * __X__ : the 6th bit is for customising DSCP values for other types. By default __X__ is set to 0. (e.g. `DSCP 37 = 10010100`)   
    * Sample DSCP patterns (is the following binary pattern a DSCP AF-value, CS-value or neither)?
        * __101010xx__: `CS=101=5 , AF=01=1 , X=0` ; Class 5 has no AF thus __Neither__ its DSCP 42
        * __010110xx__: `CS=010=2 , AF=11=3 , X=0` ; __AF23__
        * __100011xx__: `CS=100=4 , AF=01=1 , X=1` ; X=0 by in AF thus __Neither__ its DSCP 35
    * What is the binary equivalant of the follwing DSCP value?
        * __AF 41__: `CS=4=100,  AF=1=01, X=0` ; __10001000__
        * __DSCP 57__: 32+16+8+1 = __11100100__    
        
## Internal DSCP

* Switches perform something called __Internal DSCP__. A L2 frame may not contain any L3 information inside. To enable QoS in switches you need to enable QoS by __`mls qos`__ commnad in global config mode. 
* This commnad spilts the Rx and Tx queues of the interface buffer into Low, Medium and High sub-queues. Now configure the switch in the interface level, which marking goes to which queue (e.g. DSCP 0-30 = Low, 31-45 = mid, 46-63= High). 
* But, what if a raw L2 packet comes without any L3 information, there would be no DSCP to classify, then the switch uses __Internal DSCP__ which is 0 by default. You can manually mark a frame to admit in any of the queue (e.g. Vlan 7 traffic will get highest priority). 
* Use the command __`sh mls qos interface IFACE`__ to check the trust state. If it is `not_trusted` (defalut behaviour) i.e. if a Pre-marked packet comes to this interface, it will override the DSCP value to 0. 
    
```
class-map V7
    match vlan 7
policy-map TEST
    class V7
        set ip dhcp 47  !set dscp for all vlan 7 traffic
```

__IP Phones__ uses any voice traffic with default marking of __`AF53 : 1011100` = EF__ called Expedided Forwarding. Assume that there is single egress interface getting traffic from many sources, in case of congestion if the requirement is to ensure 30% bandwith for voice traffic. 
    
```
conf t
    class-map VOICE
        match dscp ef  ! if the voice traffic is matched 
    policy-map VOICE
        class VOICE
            bandwith percent 30 ! reserve 30% BW
end 
```
Now, if the VoIP traffic appears in a switch interface which has a `not-trusted` trust state, it will put the the traffic to a low priority congested queue. In such a case, best practice is to set __`mpls qos trust dscp`__ on the switch-port.

## Network Based Application Recognition (NBAR)

* Inbuild Deep Packet Inspector (DPI) in cisco IOS to understand application traffic based on its traffic profile and data-payload.  
* Useful for those protocol that has no standard port number or IP protocol number but negotiates port numbers dynamically.  Therefore not suitable for writing ACL.
* More CPU intensive
* __NBAR Modes__
    * __Passive mode__ : use `ip nbar protocol discovery` under interface mode to start monitoring traffic signature for few days to profile the channel.
    * __Active mode__ : once profiled use `match protocol PROTO` under class-map to match protocol. 
* NBAR can also match URLs. 
* Most switches do not match NBAR. 

## Modular QoS CLI (MQC)

* There is an Invisible class-map exists called __Class-Default__
    * All traffic that does not match previous class maps falls into this
    * Connot be configured/modefied using class-map global-config and it is invisible in running config.
    * Class-default has no QoS applied (FIFO)
    * You can change the QoS behaviour of the class-default by referncing it wihin a policy-map 

```
conf t
    class-map TEST1
        match access-group 101
    class-map TEST2
        match ip dscp ef
    class-map class-defaut ! ERROR well-defined class-map
    
    policy-map TEST
        class class-default 
            bandwith percent 30 ! all un-classified traffic will get 10% BW 
    
    int IFACE
        service-policy out TEST
end 
```

# Congestion Management 

* A set if QoS features that handle the queuing and scheduling of traffic. 
* The fundamental reason of having Congestion management. 
    * Default behaviour of a Queue is FIFO.
    * Incomming bursts can be bad causing congestion of queues. 
    * FIFO = No control over re-ordering of queues.
    * Congstion management brings that ordering feature. 
* Cisco IoS supports the following features 
    * Queueing: Determining which packet goes to which queues
    * Drop Policy: If a queue starts get too  full
    * Scheduling 
    * Maximum number of Queues, Not Modefiable in switches 
    * Maximum numbr of Queue length, not modefiable in switches 
* On Routers policy map that contains queuing commands can only be applied to __Outbound__ interfaces. 

## FIFO

* First in First out
* Default Queuing mechanism on all switches without QoS enabled, and most router interfaces
* Can enable it manually (pre-HQF i.e. v12.4). `no fair-quing` in interface to desable. 
* define depth of queue : `hold-queue <0-240000> [in|out]` at the interface. 
* Verify using : `sh int IFACE | inc queue` command in a format __size/max/drop/flush__

## Fair-Queuing 

* Default for any Service-Policy applied. 
* Placing different type of traffic into different queues 
* Types of FQ
    * Flow-based weighted fair queuing (WFC): Assign weight to traffic based on IP Prec
    * Class based WFC (CBWFC)
    * Low latency Queuing (LLQ)